# TP TEXT MINING

L'algorithme $Naïve$ $Bayes$ est une méthode plus ou moins intuitive qui s'appuie sur les probabilités qu'un attribut puisse appartenir à une classe spécifique pour faire une prédiction. C'est une méthode d'apprentissage supervisé plutôt aisée à mettre en place puisque l'on suppose que chaque attribut d'une classe est indépendant des autres.

Ce classifieur porte le nom de Bayes car elle repose sur le principe des probabilités conditionnelles développées par Bayes au XVIIIème siècle. En multipliant toutes les probabilités conditionnelles pour chaque attribut d'une classe donnée, on obtient la probablité qu'une nouvelle instance appartienne à cette classe. 


In [1]:
%matplotlib inline
import os.path as op
import numpy as np
import string
import math
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings

from collections import Counter
from collections import OrderedDict
from IPython.display import display, Image
from sklearn.base import BaseEstimator, ClassifierMixin
from pylab import *

warnings.filterwarnings('ignore')

Avant de commencer on récupère les fichiers `.txt` qui contiennent les critiques de films.

In [2]:
print("Loading dataset")

from glob import glob
path = 'D:/WORK/Big_Data/MDI343/'

filenames_neg = sorted(glob(op.join(path, 'data_TextMining', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join(path, 'data_TextMining', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]

Loading dataset


On rassemble tous les documents 'positifs' et 'négatifs' dans une seule liste `texts`

In [3]:
texts = texts_neg + texts_pos

On construit un vecteur de labels $y$ ayant pour valeur $0$ pour les documents `texts_neg` et $1$ pour les autres documents.

In [4]:
y = np.ones(len(texts), dtype=np.int)
#
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))
print(y) #liste des labels des documents
labels = set(y)
print(labels)

2000 documents
[0 0 0 ..., 1 1 1]
{0, 1}


**Question 1** : Implémentation du **Wordcount**. On veut compter le nombre d'occurrence de tous les mots dans tous les documents. Le but est de stocker le résultat dans une matrice qui aura comme dimension (le nombre de documents)x(le nombre de mots sans répétition). Chaque composante de la matrice sera l'occurrence d'un mot dans un document particulier. On va également construire le dictionnaire `vocabulary` qui prend comme clé un mot et en valeur l'indice de la colonne dans `counts` définie par ce mot. On peut définir également un set `word` qui listera de façon unique tous les mots de tous les documents.

Dans un premier temps on définit une fonction `count_word` qui sert à compter les occurrences d'un mot dans un document. Elle retourne une array de paires (mot, occurrence).

In [5]:
def count_word(doc):
    vocabulary = {}#dictionnaire contenant un mot du document et son indice colonne dans la matrice count
    words = set(' '.join(doc).split())#liste sans répétition de mots du documents
    n_samples = len(doc)
        
    for j,w in enumerate(words):
        vocabulary[w] = j
        
    n_features = len(words)
    counts = np.zeros((n_samples,n_features))
    
    for index_text, text in enumerate(doc):
        for w in text.split():
            counts[index_text, vocabulary[w]] += 1
    return vocabulary, counts

On peut également définir une fonction qui va nettoyer les données: enlever toute la ponctuation et les sauts de ligne.

In [6]:
def clean_data(doc):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text_clean = []
    for d in doc:
        d = regex.sub(' ', d)
        d = re.sub('\n', '', d)
        text_clean.append(d)
    return text_clean

Test de la fonction `clean_data`:

In [7]:
clean_data(texts[0:2])

['plot   two teen couples go to a church party   drink and then drive   they get into an accident   one of the guys dies   but his girlfriend continues to see him in her life   and has nightmares   what s the deal   watch the movie and   sorta   find out       critique   a mind fuck movie for the teen generation that touches on a very cool idea   but presents it in a very bad package   which is what makes this review an even harder one to write   since i generally applaud films which attempt to break the mold   mess with your head and such   lost highway   memento     but there are good and bad ways of making all types of films   and these folks just didn t snag this one correctly   they seem to have taken this pretty neat concept   but executed it terribly   so what are the problems with the movie   well   its main problem is that it s simply too jumbled   it starts off   normal   but then downshifts into this   fantasy   world in which you   as an audience member   have no idea what 

Test de la fonction `count_word`:

In [8]:
test0 = ['zéro plus zéro égale la tête à toto', 'mais zara moins zara n\' égale pas la tête à tata']
test0 = clean_data(test0)
res0 = count_word(test0)
print(res0)

({'moins': 0, 'pas': 1, 'zéro': 2, 'tata': 3, 'mais': 4, 'zara': 5, 'la': 7, 'plus': 9, 'tête': 10, 'n': 6, 'toto': 8, 'à': 11, 'égale': 12}, array([[ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.,  2.,  1.,  1.,  0.,  0.,  1.,  1.,  1.]]))


On teste la fonction `count_word` sur une partie des documents sans que les données soient nettoyées dans un premier temps (c'est-à-dire la ponctuation et les caractères spéciaux sont inlcus):

In [9]:
#on teste sur une partie des documents
test = texts[0:2]
voc, count = count_word(test)
print(voc)
print('====================')
print(count)


{'disappearances': 214, 'mold': 0, 'bringing': 1, 'excites': 144, 'i': 217, 'entire': 2, 'blair': 218, 'themselves': 3, 'work': 219, 'assuming': 4, 'tons': 327, "bastard's": 367, 'seem': 5, 'part': 216, 'jumbled': 221, 'robots': 222, 'mind': 223, '.': 6, 'has': 225, 'point': 226, 'what': 36, 'nightmares': 8, 'its': 9, 'world': 228, 'makes': 229, 'within': 11, 'y2k': 12, 'chopped': 13, 'wrapped': 15, 'movie': 195, 'accident': 16, 'parts': 44, 'starring': 232, 'arrow': 17, 'movies': 18, 'came': 234, 'engaging': 19, 'memento': 235, 'street': 236, 'is': 432, 'weird': 354, 'redundant': 21, 'entertain': 22, "doesn't": 220, 'five': 437, 'that': 23, 'review': 238, 'sense': 25, 'explanation': 26, 'middle': 240, 'nowhere': 27, 'people': 28, 'halloween': 29, 'up': 30, 'then': 241, 'lost': 31, 'stir': 32, 'decent': 33, 'folks': 242, 'took': 34, 'someone': 243, 'william': 224, 'kind': 244, 'skip': 245, "someone's": 342, 'offering': 428, 'jamie': 35, 'same': 246, 'personally': 374, '4/10': 247, 'hal

On s'intéresse à l'ensemble des documents maintenant:

In [10]:
start = time.time()
texts1 = clean_data(texts) 
vocabulary1, X1 = count_word(texts1)
end = time.time()
print('duration:', end-start)
print(X1.shape)         

duration: 1.238551378250122
(2000, 39443)


**Question 2** : Les documents ont été classés selon certaines notations: étoiles, lettres ou notes sur 5 ou sur 10. Pour chaque système d'évaluation, un seuil critique a été défini permettant de classer un document selon qu'il est au-dessus (positif) ou en dessous de ce seuil (négatif).

**Question 3 **: On cherche à implémenter un classifieur de type $Naïve$ $Bayes$ avec l'algorithme suivant:
![title](/notebooks/algoBayes.jpg "Algo Bayes")

In [11]:
class NB0(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        self.X_ = X
        self.y_ = y   
        n_samples, n_features = X.shape

        # Calcul des probabilités conditionnelles
        T = np.zeros((n_features, 2))
        T[:, 0] = np.sum(X[y == 0], axis=0) + 1 
        T[:, 1] = np.sum(X[y == 1], axis=0) + 1
        T /= np.sum(T, axis=0)
        self.prior_ = [(len(np.where(y==c)[0])*1.0)/ n_samples for c in [0, 1]]
        self.condprob_ = T
        return self

    def predict(self, X):
        n_samples = X.shape[0]
        score = np.zeros((n_samples, 2))
        score[:, 0] = np.log(self.prior_[0])
        score[:, 1] = np.log(self.prior_[1])
        
        for i in range(n_samples):
            # Pour chaque doc, calcul de la proba d'appartenir à la classe c
            score[i, :] += np.sum(np.log(self.condprob_[X[i, :] != 0]), axis=0)
        return np.argmax(score, axis=1)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

On teste le classifieur ainsi implémenté sur un dataset de petite dimension (cf vidéo Youtube https://www.youtube.com/watch?v=km2LoOpdB3A). On choisit le label ${1}$ pour la classe **Chinese** et le label ${0}$ pour la classe **Japan**. On veut pourvoir prédire la classe du groupe de mots `['beijing tokyo tokyo japan','japan hokkaido tokyo macao', 'macao shanghai chinese tokyo']` :

In [12]:
data = ['chinese beijing chinese','chinese chinese shanghai','chinese macao','tokyo japan chinese','hokkaido japan japan']
X_china = count_word(data)[1]
voc_china = count_word(data)[0]
y_china = np.array([1,1,1,0,0])
print(X_china)
print(voc_china)


[[ 2.  0.  0.  0.  0.  0.  1.]
 [ 2.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.]
 [ 1.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  2.  0.  0.  0.]]
{'shanghai': 5, 'chinese': 0, 'tokyo': 1, 'hokkaido': 2, 'japan': 3, 'beijing': 6, 'macao': 4}


In [13]:
nb = NB0()
nb.fit(X_china, y_china)


NB0()

In [14]:
data_test = ['beijing tokyo tokyo japan','japan hokkaido tokyo macao', 'macao shanghai chinese tokyo']
X_test_china = count_word(data_test)[1]
voc_test_china= count_word(data_test)[0]
predict_china = nb.predict(X_test_china)
print(predict_china)

[0 0 1]


Le test précédent est concluant sur un petit dataset. On peut passer au test proposé dans l'énoncé sur les données nettoyées ($ie$ sans la ponctuation) .

In [15]:
print(X1.shape)
start = time.time()
nb.fit(X1[::2], y[::2])
predict_texts1 = nb.predict(X1[1::2])
end = time.time()
print('duration:', end-start)
print(predict_texts1)


(2000, 39443)
duration: 0.3694641590118408
[0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 

In [16]:
start = time.time()
score_texts1 = nb.score(X1[1::2], y[1::2])
end = time.time()
print('duration:', end-start)
print('le score obtenu pour les data traitées:', score_texts1)

duration: 0.14691495895385742
le score obtenu pour les data traitées: 0.82


On peut tenter de faire la même chose sur les data brutes :

In [17]:
start = time.time()
vocabulary_raw, X_raw= count_word(texts)
end = time.time()
print('duration:', end-start)
print(X_raw.shape)

duration: 1.230593204498291
(2000, 50920)


In [18]:
start = time.time()
nb.fit(X_raw[::2], y[::2])
predict_texts = nb.predict(X_raw[1::2])
end = time.time()
print('duration:', end-start)
print(predict_texts)
print('nb de documents traités:', len(predict_texts))

duration: 0.4640522003173828
[0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 

In [19]:
start = time.time()
score_texts = nb.score(X_raw[1::2], y[1::2])
end = time.time()
print('duration:', end-start)
print('le score obtenu pour les data brutes:', score_texts)

duration: 0.16891264915466309
le score obtenu pour les data brutes: 0.836


On constate qu'on obtient un meilleur score avec les data brutes par rapport aux data nettoyées. La raison peut-être dû au fait qu'en enlevant toute la ponctuation, on n'a pas tenu compte des notes données aux films de type '9/10' ou '2/10' qui peuvent clairement être un indicateur sur la classe.    
Pour revenir à l'algorithme $Naïve$ $Bayes$ qui a été implémenté, j'ai fait les choix suivants:  
* La variable X à traiter est la matrice `count` obtenue par la fonction `count_word` et non la liste de documents d'apprentissage. Celle-ci est pré-traitée dans un premier temps pour obtenir la matrice, le vocabulaire
* La fonction `fit` permet de calculer la fréquence des classes `prior` et les probabilités conditionnelles `cond_prob` par classe de documents. Dans la relation $condprob[t][c] \leftarrow \frac{T_{ct}+1}{\sum_{t'} T_{ct'}+1}$, le terme $1$ permet d'attribuer une probabilité non nulle à des mots qui ne seraient pas dans l'ensemble d'apprentissage. Dans le classifieur $Naïve$ $Bayes$ implémenté dans `sklearn`, ce terme devient le paramètre $\alpha$ qui peut-être modulé  
* La fonction `predict` s'applique sur des données test et plus précisément la matrice `count` obtenus à partir des données test. Pour chaque document, chaque mot et pour chaque classe, on regarde la probabilité conditionnelle de ce mot à appartenir à une classe spécifique, on somme de façon logarithmique pour obtenir un score pour chaque document du test. On récupère ensuite l'$argmax$ de ce score qui va nous donner la classe la plus probable du document    

**Question 4 **: On va évaluer les performances avec une cross-validation 5-fold. Pour cela, on peut utiliser la fonction `cross_val_score` de $sklearn$

In [20]:
from sklearn.model_selection import cross_val_score
start = time.time()
scores = cross_val_score(nb,X_raw[1::2], y[1::2], cv = 5)
end = time.time()
print('duration:', end-start)
print(scores)
print('la moyenne des scores obtenus par cross_validation est:', scores.mean())
print('la déviation standard est:', scores.std())

duration: 3.7883408069610596
[ 0.855  0.825  0.78   0.81   0.78 ]
la moyenne des scores obtenus par cross_validation est: 0.81
la déviation standard est: 0.0284604989415


**Question 5 **: On modifie la fonction `count_word` qui va tenir compte des *stop-words* qui se trouve dans un fichier particulier `english.stop`  
Dans un premier temps, on récupère un set de mots uniques de `english.stop`:

In [21]:
path_stop = 'D:/WORK/Big_Data/MDI343/data_TextMining/english.stop'
stop_words = open(path_stop).read().split('\n')
print(stop_words)
print('========================================')
print('taille de la liste stop_words:', len(stop_words))

['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'conta

In [22]:
def count_word_stop(doc, stop):
    vocabulary = {}#dictionnaire contenant un mot du document et son indice colonne dans la matrice count
    words = set(' '.join(doc).split())#liste sans répétition de mots du documents
    words -= set(stop)
    
    n_samples = len(doc)
    n_features = len(words)
    
    for j,w in enumerate(words):
        vocabulary[w] = j

    counts = np.zeros((n_samples,n_features))
    for index_text, text in enumerate(doc):
        for w in text.split():
            if w in vocabulary:
                counts[index_text, vocabulary[w]] += 1
    return vocabulary, counts

Test de la fonction `count_word_stop` sur une partie de l'échantillon de documents:

In [23]:
start = time.time()
voc_stop, count_stop  = count_word_stop(test, stop_words)
end = time.time()
print('duration:', end-start)
print(voc_stop)
print('====================')
print(count_stop)

duration: 0.0
{'disappearances': 134, 'characters': 218, 'mold': 0, 'bringing': 1, 'picking': 63, 'deserted': 64, 'running': 276, 'entire': 2, 'blair': 137, 'idea': 221, 'work': 138, 'assuming': 3, 'tons': 219, "bastard's": 245, 'elm': 222, 'part': 136, 'shelves': 258, 'pink': 284, 'jumbled': 139, 'beauty': 65, 'robots': 140, 'american': 66, '.': 4, 'thing': 262, 'point': 143, 'rarely': 67, 'ago': 144, 'audience': 225, 'video': 68, 'generation': 227, 'hide': 228, 'entertaining': 230, 'back': 81, 'nightmares': 6, 'brain': 7, 'h20': 211, 'tugboat': 69, 'world': 145, 'lazy': 16, 'sutherland': 70, 'chopped': 9, 'wrapped': 11, 'make': 235, 'accident': 12, '"': 232, 'baldwin': 236, 'bad': 220, 'starring': 147, 'arrow': 13, 'movies': 14, 'hey': 72, 'engaging': 15, 'memento': 149, 'occasional': 234, 'dreams': 73, 'street': 150, 'weird': 237, 'password': 204, 'redundant': 17, 'entertain': 18, 'happy': 74, 'turn': 240, 'write': 75, 'review': 151, 'sense': 19, 'explanation': 20, 'figured': 47, 'm

In [24]:
print('taille du dictionnaire sans les mots stop:',len(voc_stop))
print('taille du dictionnaire avec les mots stop:',len(voc))


taille du dictionnaire sans les mots stop: 285
taille du dictionnaire avec les mots stop: 442


On a bien vérifié que la taille du dictionnaire est plus petite sans les *stop_words* qu'avec, ce qui parait logique.  
On regarde si les performances sont améliorées en enlevant les mots stop avec notre classifieur NB et une cross_validation de 5_fold:

In [26]:
start = time.time()
vocabulary_stop, X_stop = count_word_stop(texts, stop_words)
end = time.time()
print('duration:', end-start)
print(X_stop.shape)

duration: 1.0947341918945312
(2000, 50375)


In [27]:
start = time.time()
scores_stop = cross_val_score(nb,X_stop[1::2], y[1::2], cv = 5)
end = time.time()
print('duration:', end-start)
print(scores_stop)
print('la moyenne des scores obtenus par cross_validation est:', scores_stop.mean())
print('la déviation standard est:', scores_stop.std())

duration: 3.869823694229126
[ 0.845  0.805  0.77   0.85   0.78 ]
la moyenne des scores obtenus par cross_validation est: 0.81
la déviation standard est: 0.0327108544676


Pour conclure sur la **question 5**, on constate que le score est légérement détérioré par le fait de ne pas tenir compte des *stop_words* bien que cela reste du même ordre de magnitude.  
A priori on peut se dire que ces mots rajoutent du bruit, c'est-à-dire qu'ils ne contribuent pas de façon déterminante à la détermination d'une classe de documents.  
Cependant, on peut remarquer que certains mots de la liste *stop_words* peuvent nuancer une opinion et donc influer sur l'appartenance à une classe de tel ou tel documents. Par exemple *although*, *however* et toutes les conjonctions pouvant nuancer une opinion, les verbes à la forme négative tels que *don't* ou *couldn't* et certains adverbes comme *awfully*.  
Ce qui laisse à penser qu'il faudrait en fait créer une troisième classe d'opinion neutre et refaire l'étude avec des labels ${-1,0,1}$ pour *négatif*, *neutre* ou *positif*, ce qui permettrait éventuellement d'affiner le modèle.

In [28]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline


## Pour aller plus loin... Utilisation de $sklearn$ 


**Question 1**:
On compare notre classifieur $NB0$ avec celui de $sklearn$. Pour ce faire, on va utiliser la classe `CountVectorizer` et un `Pipeline`:

In [29]:
pipeline = Pipeline([('vectorizer', CountVectorizer()), ('nb_sk', MultinomialNB())])
start = time.time()
scores_sk = cross_val_score(pipeline, texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_sk)
print('la moyenne des scores obtenus par cross_validation est:', scores_sk.mean())
print('la déviation standard est:', scores_sk.std())

duration: 5.0367231369018555
[ 0.81   0.825  0.81   0.825  0.79 ]
la moyenne des scores obtenus par cross_validation est: 0.812
la déviation standard est: 0.0128840987267


On constate qu'on obtient à peu de chose près les mêmes résultats que notre propre classifieur, bien que la déviation standard soit plus petite dans ce cas.  
On avait obtenu précédemment :  
[ 0.855  0.825  0.78   0.81   0.78 ]  
la moyenne des scores obtenus par cross_validation est: 0.81  
la déviation standard est: 0.0284604989415  

On s'intéresse maintenant aux bigrammes (cf. http://scikit-learn.org/stable/modules/feature_extraction.html). On va utiliser le paramètre `ngram_range` de `CountVectorizer`:

In [30]:
pipeline_bi = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))), ('nb_sk', MultinomialNB())])
start = time.time()
scores_bi = cross_val_score(pipeline_bi, texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_bi)
print('la moyenne des scores obtenus par cross_validation est:', scores_bi.mean())
print('la déviation standard est:', scores_bi.std())

duration: 23.918751001358032
[ 0.8175  0.8375  0.8225  0.8425  0.8325]
la moyenne des scores obtenus par cross_validation est: 0.8305
la déviation standard est: 0.0092736184955


Le score obtenu est bien meilleur en prenant en compte les bigrammes. En effet (et je reprends la définition de **Wikipédia** ), un bigramme est une séquence de éléments adjacents d'une chaine de token (lettres, syllabes ou mots). La fréquence de distribution d'un bigramme dans une chaine permet de déterminer la probabilité d'un token, connaissant le token précédent. 

On regarde maintenant les sous chaines de caractères et on ré-itère l'opération:

In [32]:
pipeline_char = Pipeline([('vectorizer', CountVectorizer(analyzer='char', ngram_range=(3, 6))), ('nb_sk', MultinomialNB())])
start = time.time()
scores_char = cross_val_score(pipeline_char, texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_char)
print('la moyenne des scores obtenus par cross_validation est:', scores_char.mean())
print('la déviation standard est:', scores_char.std())

duration: 153.97074460983276
[ 0.8225  0.825   0.8175  0.845   0.7975]
la moyenne des scores obtenus par cross_validation est: 0.8215
la déviation standard est: 0.0152151240547


Le score est légèrement moins bon qu'avec les bigrammes, dans la mesure où en analysant par chaines caratères(découpées par longueur de 3 à 6), on n'a plus la même organisation du texte qui a son importance. Le score est complètement dégradé si on ne précise pas la longueur des chaines de caractères à prendre en compte dans l'analyse (de l'ordre de 60%)

**Question 2 ** : On va tester d'autres algorithmes.  
Dans un premier temps on s'intéresse à la régression logistique qui est un modèle de régression binômiale. On garde l'analyse en bigrammes.

In [33]:
from sklearn.linear_model import LogisticRegression
pipeline_logit = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))), ('logistic', LogisticRegression())])
start = time.time()
scores_logit = cross_val_score(pipeline_logit, texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_logit)
print('la moyenne des scores obtenus par cross_validation est:', scores_logit.mean())
print('la déviation standard est:', scores_logit.std())

duration: 31.086959838867188
[ 0.8225  0.8525  0.8525  0.87    0.865 ]
la moyenne des scores obtenus par cross_validation est: 0.8525
la déviation standard est: 0.0165075740192


Avec la régression logistique, on a un excellent score comparé au Naïve Bayes dans la mesure où la régression logistique permet de tenir compte des dépendances entre les attributs, ce qui peut être le cas dans une analyse sémantique.

On teste maintenant les supports vector machine:

In [34]:
from sklearn import svm
pipeline_svm = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))), ('linearSVC', svm.LinearSVC())])
start = time.time()
scores_svm = cross_val_score(pipeline_svm, texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_svm)
print('la moyenne des scores obtenus par cross_validation est:', scores_svm.mean())
print('la déviation standard est:', scores_svm.std())


duration: 34.111642599105835
[ 0.8175  0.845   0.8475  0.87    0.87  ]
la moyenne des scores obtenus par cross_validation est: 0.85
la déviation standard est: 0.0194293592277


On obtient un résultat similaire à celui de la régression logistique.

**Question 3** : 
On va utiliser la librairie NLTK pour procéder à une *racinisation*. D'après **Wikipédia**, c'est un procédé de transformation des flexions en leur racine: la racine d’un mot correspond à la partie du mot restante une fois que l’on a supprimé son préfixe et suffixe.

In [35]:
import nltk
from nltk import SnowballStemmer
stemmer = SnowballStemmer('english')

def stem_data(doc): 
    all_stem =[]
    for index_text, text in enumerate(doc):
        text_stem = []
        for w in text.split():
            text_stem.append(stemmer.stem(w))
        s = ' '.join(text_stem)
        all_stem.append(s) 
    return all_stem


Test de la fonction `stem_data` sur une partie des data:

In [36]:
start = time.time()
stem_texts_s = stem_data(texts[:2])
end = time.time()
print('duration:', end-start)
print(stem_texts_s)

duration: 0.0157926082611084
['plot : two teen coupl go to a church parti , drink and then drive . they get into an accid . one of the guy die , but his girlfriend continu to see him in her life , and has nightmar . what the deal ? watch the movi and " sorta " find out . . . critiqu : a mind-fuck movi for the teen generat that touch on a veri cool idea , but present it in a veri bad packag . which is what make this review an even harder one to write , sinc i general applaud film which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad way of make all type of film , and these folk just didn\'t snag this one correct . they seem to have taken this pretti neat concept , but execut it terribl . so what are the problem with the movi ? well , it main problem is that it simpli too jumbl . it start off " normal " but then downshift into this " fantasi " world in which you , as an audienc member , have no idea what go on . there are d

Après ce test, on applique la fonction à l'ensemble des documents:

In [37]:
start = time.time()
stem_texts = stem_data(texts)
end = time.time()
print('duration:', end-start)
print(len(stem_texts))

duration: 15.373677730560303
2000


In [38]:
start = time.time()
voc_stem, count_stem = count_word(stem_texts)
end = time.time()
print('duration:', end-start)
print('taille du dictionnaire de stem:',len(voc_stem))
print('taille du dictionnaire data brutes:', len(vocabulary_raw))

duration: 1.1214759349822998
taille du dictionnaire de stem: 33294
taille du dictionnaire data brutes: 50920


In [39]:
pipeline_stem = Pipeline([('vectorizer', CountVectorizer()), ('nb_sk', MultinomialNB())])
start = time.time()
scores_stem = cross_val_score(pipeline_stem, stem_texts, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_stem)
print('la moyenne des scores obtenus par cross_validation est:', scores_stem.mean())
print('la déviation standard est:', scores_stem.std())

duration: 4.520931005477905
[ 0.795   0.815   0.8     0.8325  0.8025]
la moyenne des scores obtenus par cross_validation est: 0.809
la déviation standard est: 0.0134721935853


Par rapport au classifieur NB comprenant les bigrammes et dont le score était de 0.8305, cette implémentation est un peu moins bonne, bien que la déviation standard soit meilleure.

**Question 4**: On va filtrer les mots par catégorie grammaticale (POS : Part Of Speech) et ne garder que les noms NM, les verbes V, les adverbes ADV et les adjectifs ADJ pour la classiﬁcation. 

In [64]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk import word_tokenize,sent_tokenize

def token_tag(doc):
    list_to_keep =['JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'RB', ' RBR',
                'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    all_tags = []
    for index_text, text in enumerate(doc):
        text_tag = []
        tag_tokeep = []
        for w in text.split():
            text_tag.append(w)
        tags = pos_tag(text_tag)
        tag_tokeep = [t[0] for t in tags if t[1] in list_to_keep]
        s = ' '.join(tag_tokeep)
        all_tags.append(s) 
    return all_tags


Test de la fonction `token_tag` sur une partie des documents: 

In [66]:
print(token_tag(test))
print(len(token_tag(test)))

['plot teen couples go church party drink then drive get accident guys dies girlfriend continues see life has nightmares what\'s deal watch movie " sorta " find critique mind-fuck movie teen generation touches very cool idea presents very bad package is makes review even write i generally applaud films attempt break mold mess head such lost highway memento are good bad ways making types films folks just didn\'t snag correctly seem have taken pretty neat concept executed terribly are problems movie well main problem is it\'s simply too jumbled starts " normal " then downshifts " fantasy " world audience member have idea what\'s going are dreams are characters coming back dead are others look dead are strange apparitions are disappearances are looooot chase scenes are tons weird things happen most is simply not explained now i personally don\'t mind trying unravel film now then does is give same clue over again i get kind fed while is film\'s biggest problem it\'s obviously got big secre

In [67]:
start = time.time()
token_text = token_tag(texts)
end = time.time()
print('duration:', end-start)
print(len(token_text))

duration: 65.42023015022278
2000


On applique ensuite le pipeline (CountVectorizer, MultinomialNB) au data auxquelles on a appliqué le filtre de catégorisation grammaticale):

In [68]:
pipeline_pos = Pipeline([('vectorizer', CountVectorizer()), ('nb_sk', MultinomialNB())])
start = time.time()
scores_pos = cross_val_score(pipeline_pos, token_text, y, cv=5)
end = time.time()
print('duration:', end-start)
print(scores_pos)
print('la moyenne des scores obtenus par cross_validation est:', scores_pos.mean())
print('la déviation standard est:', scores_pos.std())

duration: 3.3860716819763184
[ 0.8025  0.8175  0.81    0.835   0.7925]
la moyenne des scores obtenus par cross_validation est: 0.8115
la déviation standard est: 0.0143701078632


### Conclusion:  
  
Pour conclure ce TP, on voit que les différents algorithmes renvoient sensiblement les mêmes scores. Certains traitements de données peuvent détériorer le score (comme un pré-traitement pour enlever la ponctuation par exemple). La régression logistique a obtenu le meilleur score suivi de la SVM mais le Naive Bayes reste un bon classifieur.
  
|Algorithme                             | Temps d'execution (s)| Score | std   |
| ------------------------------------- |:--------------------:| -----:|------:|
| NB home_made                          | 2.51                 | 0.836 | NaN   |
| NB home_made, cross_val               | 3.78                 |   0.81|0.02846|
| NB home_made, cross_val, stop_word    | 4.97                 |   0.81|0.03271|
| NB skl, cross_val                     | 5.03                 |  0.812|0.01288|
| NB skl, cross_val, bigrammes          | 23.92                | 0.8305|0.00927|
| NB skl, cross_val, char               | 153.9707             | 0.8215|0.01522|
| Logit, cross_val, bigrammes           | 31.09                | 0.8525|0.01651|
| SVM, cross_val, bigrammes             | 34.1                 |  0.85 |0.01942|
| NB skl, cross_val, Snowball           | 21.01                | 0.809 |0.01347|
| NB skl, cross_val, POS                | 68.81                | 0.8115|0.01437|